In [15]:
import numpy as np
import pandas as pd
from scipy.io import loadmat

In [16]:
 # machine learning libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
#import xgboost as xgb

(185,)

In [17]:
from scipy.io import loadmat

badTrials = loadmat('bad_trials_OFC.mat')
badTrials

#subjects except for subject 8 & 9 don't match for trial #s
#behavior some are timed out 

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Thu Sep 27 15:27:57 2018',
 '__version__': '1.0',
 '__globals__': [],
 'bad_trials_OFC': array([[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}

# doing something a little weird now. take mean of each electrode's value for each round. Maybe this'll be helpful for a neural net ?

In [18]:
electrode_dict = dict()

# loop over subjects
for i in range(1, 11):
    str_int = str(i)
    #print(str_int)
    if i < 10:
        str_int = '0' + str_int
    
    tempNeur = loadmat('ephys_data/s' + str_int + '_ofc_hg_events.mat')['buttonpress_events_hg'][:, 0:951, :]
    # loop over rounds
    starter = [np.mean(i) for i in tempNeur[0, :, :]]
    
    for r in range(1, tempNeur.shape[0]):
        meanArray = [np.mean(i) for i in tempNeur[r, :, :]]
        starter = np.vstack((starter, meanArray))
    electrode_dict[str(i)] = starter


In [19]:
i = 10
str_int = str(i)
#print(str_int)
if i < 10:
    str_int = '0' + str_int

tempNeur = loadmat('ephys_data/s' + str_int + '_ofc_hg_events.mat')['buttonpress_events_hg'][:, 0:951, :]
# loop over rounds
starter = [np.mean(i) for i in tempNeur[0, :, :]]

for r in range(1, tempNeur.shape[0]):
    meanArray = [np.mean(i) for i in tempNeur[r, :, :]]
    starter = np.vstack((starter, meanArray))
electrode_dict[str(i)] = starter

In [20]:
for i in range(1, 11):
    str_int = str(i)
    print(electrode_dict['str_int'].shape)

KeyError: 'str_int'

In [33]:
import os
import pickle
with open(os.path.join('electrode_mean_dict.pickle'), 'wb') as handle1:
    pickle.dump(electrode_dict, handle1, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data_filtered = pd.read_csv('Lillian_iter1/data567.csv')

In [21]:
data_filtered.head()

,Unnamed: 0,Electrode0_avg_bin1,Electrode0_avg_bin2,Electrode0_avg_bin3,Electrode0_max_bin1,Electrode0_max_bin2,Electrode0_max_bin3,Electrode0_min_bin1,Electrode0_min_bin2,Electrode0_min_bin3,...,buttonpress.time,choice.class,choice.location,choice.time,conf.time,newround.time,outcome,reveal.time,round,subject_number
0,0,-1.225318,-4.017107,-1.020703,9.759506,-1.117182,7.899104,-7.119076,-8.019692,-5.910308,...,8.003,Gamble,Left,6.957,8.019,6.191,Win,8.535,1,1
1,1,1.255444,-4.530762,-3.141041,31.750257,1.794326,7.294234,-7.611065,-9.168185,-9.389197,...,11.441,Gamble,Left,10.316,11.457,9.550,Win,11.972,2,1
2,6,-2.316165,-1.515046,-2.186162,4.166417,6.010494,3.413128,-8.969111,-8.968894,-7.393499,...,28.753,Gamble,Right,27.472,28.769,26.707,Win,29.285,7,1
3,22,-0.240522,-2.765261,-1.707884,5.863612,3.445650,3.271917,-6.099566,-5.520102,-5.765873,...,86.972,Safebet,Left,85.925,86.988,85.160,WouldHaveLost,87.503,23,1
4,38,2.264580,0.770095,2.598170,8.038921,5.173100,9.414794,-2.865637,-3.738700,-4.209744,...,146.644,Gamble,Right,145.628,146.675,144.863,Win,147.191,39,1


In [31]:
data_filtered[data_filtered['subject_number'] == 1]['round'].unique()

In [21]:
y_dict = dict()
x_dict = dict()

game_model = pd.read_csv('behavioral_data/gamble_choices.csv')

for i in range(1,10):
    str_int = str(i)
    if i < 10:
        str_int = '0' + str_int
    tempBehav = pd.read_csv('behavioral_data/gamble.data.s' + str_int + '.csv')
    tempNeur = electrode_dict[str(i)]
    tempBehav = tempBehav.merge(game_model, left_index=True, right_index=True)

    no_bad = tempBehav[badTrials['bad_trials_OFC'][i-1] == 0]
    onlyThreshold = no_bad[no_bad['choice.location'] != 'Timeout']
    
    bool_array = [(onlyThreshold['Risky.bet.hidden.number'] == 5).tolist() 
                  or (onlyThreshold['Risky.bet.hidden.number'] == 6).tolist() 
                  or (onlyThreshold['Risky.bet.hidden.number'] == 7).tolist()][0]
    
    y_dict[str(i)] = onlyThreshold[bool_array]['choice.class']
    x_dict[str(i)] = tempNeur[bool_array]
    

In [22]:
# oops forgot to do 10
i = 10
str_int = str(i)
if i < 10:
    str_int = '0' + str_int
tempBehav = pd.read_csv('behavioral_data/gamble.data.s' + str_int + '.csv')
tempNeur = electrode_dict[str(i)]
tempBehav = tempBehav.merge(game_model, left_index=True, right_index=True)

no_bad = tempBehav[badTrials['bad_trials_OFC'][i-1][0:138] == 0]
onlyThreshold = no_bad[no_bad['choice.location'] != 'Timeout']

bool_array = [(onlyThreshold['Risky.bet.hidden.number'] == 5).tolist() 
              or (onlyThreshold['Risky.bet.hidden.number'] == 6).tolist() 
              or (onlyThreshold['Risky.bet.hidden.number'] == 7).tolist()][0]

#print(tempNeur.shape)


y_dict[str(i)] = onlyThreshold[bool_array]['choice.class']
x_dict[str(i)] = tempNeur[bool_array]




In [23]:
# stack all the stuff
#starter_y = y_dict['1']
starter_x = x_dict['1']
for i in range(2, 11): 
    #starter_y = np.vstack((starter_y, y_dict[str(i)]))
    starter_x = np.vstack((starter_x, x_dict[str(i)]))

In [24]:
y_starter = y_dict['1']
for i in range(2, 11): 
    y_starter = np.append(y_starter, y_dict[str(i)])
    
y_starter

array(['Gamble', 'Gamble', 'Gamble', 'Safebet', 'Gamble', 'Safebet',
       'Safebet', 'Safebet', 'Safebet', 'Gamble', 'Safebet', 'Gamble',
       'Gamble', 'Gamble', 'Gamble', 'Gamble', 'Gamble', 'Gamble',
       'Gamble', 'Gamble', 'Safebet', 'Gamble', 'Gamble', 'Gamble',
       'Gamble', 'Safebet', 'Gamble', 'Gamble', 'Safebet', 'Gamble',
       'Gamble', 'Safebet', 'Gamble', 'Safebet', 'Safebet', 'Gamble',
       'Safebet', 'Gamble', 'Gamble', 'Gamble', 'Safebet', 'Gamble',
       'Gamble', 'Safebet', 'Gamble', 'Safebet', 'Gamble', 'Gamble',
       'Safebet', 'Safebet', 'Gamble', 'Gamble', 'Safebet', 'Gamble',
       'Gamble', 'Safebet', 'Safebet', 'Gamble', 'Safebet', 'Safebet',
       'Safebet', 'Safebet', 'Safebet', 'Gamble', 'Gamble', 'Gamble',
       'Safebet', 'Gamble', 'Safebet', 'Gamble', 'Gamble', 'Gamble',
       'Gamble', 'Gamble', 'Gamble', 'Safebet', 'Gamble', 'Gamble',
       'Safebet', 'Gamble', 'Gamble', 'Safebet', 'Gamble', 'Gamble',
       'Gamble', 'Gamble', 'Gam

In [26]:
starter_y = []
for i in y_starter:
    if i == 'Gamble':
        temp_int = 1
    else:
        temp_int = 0
    starter_y.append(temp_int)

In [27]:
starter_y = np.asarray(starter_y)


In [170]:
type(starter_x[0, :])

numpy.ndarray

In [182]:
a = np.array([[1, 1], [2, 2], [3, 3]])
np.insert(a, 2, 5, axis = 1)
all_data = np.insert(starter_x.tolist(), [951], starter_y.tolist(), axis = 1)

# WHY CAN'T YOU JUST BE NORMAL

In [185]:
b = np.empty(0)
b

array([], dtype=float64)

In [186]:
all_data2 = np.append(starter_x[0, :], starter_y[0])

for i in range(1, starter_x.shape[0]):
    temp_row = np.append(starter_x[i, :], starter_y[i])
    all_data2 = np.vstack((all_data2, temp_row))
    
all_data2.shape

(185, 952)

In [183]:
all_data.shape
#starter_x.shape
#starter_y.shape



(185, 1136)

[array([[ 1.2378188 ,  1.1902817 ,  1.1404425 , ..., -4.445043  ,
         -4.4493484 , -4.4478025 ],
        [-3.063423  , -3.163988  , -3.2417045 , ..., -0.37556434,
         -0.44466895, -0.52591854],
        [-4.4520445 , -4.458924  , -4.4558935 , ..., -0.21742985,
         -0.0125622 ,  0.19203702],
        ...,
        [ 2.4871254 ,  2.5225816 ,  2.5466688 , ..., -2.6953747 ,
         -2.7284272 , -2.7709694 ],
        [-1.3781619 , -1.3377264 , -1.2977856 , ..., -4.6430326 ,
         -4.627326  , -4.603806  ],
        [ 3.8336878 ,  3.8229616 ,  3.8020382 , ...,  3.9479809 ,
          3.897419  ,  3.8406801 ]], dtype=float32),
 array([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
        1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 

In [55]:
from sklearn.utils import resample

bootX, bootY = resample(starter_x, starter_y, replace=True, n_samples=500, random_state=1)
#bootY = resample(starter_y, replace=True, n_samples=500, random_state=1)


In [69]:
import os
import pickle
with open(os.path.join('bootX.pickle'), 'wb') as handle1:
    pickle.dump(bootX, handle1, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join('bootY.pickle'), 'wb') as handle1:
    pickle.dump(bootY, handle1, protocol=pickle.HIGHEST_PROTOCOL)

In [227]:
np.asarray(y_data)

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,

In [29]:
X_train, X_test, y_train, y_test = train_test_split(starter_x, starter_y, test_size=0.2, random_state=100)

In [31]:
from sklearn.utils import resample

bootX_train, bootY_train = resample(X_train, y_train, replace=True, n_samples=400, random_state=100)
bootX_test, bootY_test = resample(X_train, y_train, replace=True, n_samples=100, random_state=100)

# neural net 

In [3]:
import os
import pickle
with open(os.path.join('x_data_neural.pickle'),'rb') as handle1:
    x_data = pickle.load(handle1)
    
with open(os.path.join('y_data_neural.pickle'),'rb') as handle2:
    y_data = pickle.load(handle2)

(500, 951)

In [5]:
with open(os.path.join('bootX.pickle'),'rb') as handle3:
    boot_x = pickle.load(handle3)
    
with open(os.path.join('y_data_neural.pickle'),'rb') as handle4:
    boot_y = pickle.load(handle4)

In [10]:
import tensorflow as tf
tf.reset_default_graph()

In [8]:
'''x_data = boot[:, 0:951]
y_data = np.asarray([int(i) for i in boot[:, 951]])'''
X_train, X_test, y_train, y_test = train_test_split(boot_x, boot_y, test_size=0.2, random_state=100)

In [11]:
# define features
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(bootX_train)

# dense neural network classifier
# two layers 300 and 100
# 10 clases
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes=10,
                                         feature_columns=feature_cols)

# if TensorFlow >= 1.1, make compatible with sklearn
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) 

Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please convert numpy dtypes explicitly.
Instructions for updating:
Please specify feature columns explicitly.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a3e6fa4a8>, '_master': '', '

In [12]:
# fit the model, 4000 iterations
dnn_clf.fit(X_train, y_train, batch_size=50, steps=1000)

Instructions for updating:
Please feed input to tf.data to support dask.
Instructions for updating:
Please access pandas data directly.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/kv/syd7xj955mn4794h4_kc40jw0000gn/T/tmpbyglxe8s/model.ckpt.
INFO:tensorflow:loss = 7.1705537, step = 1
INFO:tensorflow:global_step/sec: 34.3716
INFO:tensorflow:loss = 0.22614253, step = 101 (2.920 sec)
INFO:tensorflow:global_step/sec: 55.3606
INFO:tensorflow:loss = 0.008030944, step = 201 (1.798 sec)
INFO:tensorflow:global_step/sec: 60.3546
INFO:tensorflow:loss = 0.0047698473, step = 301 (1.656 sec)
INFO:tensorflow:global_step/sec: 71.3169
INFO:tensorflow:loss = 0.00

SKCompat()

In [13]:
# Calculate accuracies
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test)
print()
print('Accuracy',accuracy_score(y_test, y_pred['classes']))

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/kv/syd7xj955mn4794h4_kc40jw0000gn/T/tmpbyglxe8s/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Accuracy 0.96


In [75]:
type(starter_x)

numpy.ndarray

(500, 951)

In [42]:
import tensorflow as tf

# Reset graph
tf.reset_default_graph()


In [57]:
tf.reset_default_graph()

In [58]:
n_inputs = 951  
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 2

In [59]:
# Placeholders for data (inputs and targets)
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [60]:
# Define neuron layers (ReLU in hidden layers)
# We'll take care of Softmax for output with loss function

def neuron_layer(X, n_neurons, name, activation=None):
    # X input to neuron
    # number of neurons for the layer
    # name of layer
    # pass in eventual activation function
    
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        
        # initialize weights to prevent vanishing / exploding gradients
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        
        # Initialize weights for the layer
        W = tf.Variable(init, name="weights")
        # biases
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        
        # Output from every neuron
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [61]:
# Define the hidden layers
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [62]:
# Define loss function (that also optimizes Softmax for output):

with tf.name_scope("loss"):
    # logits are from the last output of the dnn
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [63]:
with tf.name_scope("dropout"):
    dropout = tf.nn.dropout(X, 0.1)

In [64]:
# Training step with Gradient Descent

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [65]:
# Evaluation to see accuracy

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [72]:
# create validation set.
# make it 20% of train set

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=100)
# i think i'll probably have to bootstrap the data since there's so little.

In [67]:
batch_size = 30
iteration = 0
X_train2[iteration*batch_size:(iteration+1)*batch_size]

array([[ 3.2789652 ,  3.451926  ,  3.63704   , ..., -2.0682557 ,
        -2.0874434 , -2.1092384 ],
       [-1.8381251 , -1.9137055 , -1.9772305 , ...,  0.7765798 ,
         0.8461931 ,  0.9072229 ],
       [-2.3662817 , -2.3859591 , -2.4049356 , ..., -1.9866726 ,
        -2.1013465 , -2.2109008 ],
       ...,
       [-0.6212216 , -0.50722575, -0.38178694, ...,  5.507732  ,
         5.699082  ,  5.860168  ],
       [-0.70995027, -0.7249624 , -0.7513781 , ..., -1.2779174 ,
        -1.3848013 , -1.4903026 ],
       [-0.18816411, -0.23202264, -0.28108588, ...,  1.7404133 ,
         1.7071847 ,  1.6743317 ]], dtype=float32)

In [73]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 3
batch_size = 30

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size):
            X_batch = X_train2[iteration*batch_size:(iteration+1)*batch_size]
            y_batch = y_train2[iteration*batch_size:(iteration+1)*batch_size]
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_val,
                                            y: y_val})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt") # save model

0 Train accuracy: 0.6785714 Val accuracy: 0.56666666
1 Train accuracy: 0.78571427 Val accuracy: 0.53333336
2 Train accuracy: 0.85714287 Val accuracy: 0.4


In [126]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt
Predicted classes: [0 1 0 0 0 1 1 0 0 1 1 1 0 1 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0]
Actual classes:    [1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 1 1]


In [129]:
sum(y_pred == y_test)/len(y_test)

0.43243243243243246

# neural net 2 with result_frame

In [ ]:
result_frame2 = result_frame.drop(['Subject'], axis = 1)
result_frame2 = result_frame2.drop('Unnamed: 0', axis = 1)

result_frame2.head()
X_data = result_frame2.drop('Result', axis = 1)
y_data = result_frame2['Result']

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=100)

In [ ]:
n_inputs = 10 
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 2

In [162]:
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=100)


In [164]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 5
batch_size = 30

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size):
            X_batch = X_train2[iteration*batch_size:(iteration+1)*batch_size]
            y_batch = y_train2[iteration*batch_size:(iteration+1)*batch_size]
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_val,
                                            y: y_val})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt") # save model

ValueError: could not convert string to float: '[-3.776883363723755, -3.7319576740264893, -3.66705322265625, -3.5826196670532227, -3.479315757751465, -3.3579912185668945, -3.2196598052978516, -3.0654656887054443, -2.896655797958374, -2.7145423889160156, -2.5204687118530273, -2.315775156021118, -2.1017649173736572, -1.8796709775924683, -1.6506301164627075, -1.4156547784805298, -1.175615668296814, -0.9312250018119812, -0.6830260753631592, -0.43139466643333435, -0.17653672397136688, 0.08149821311235428, 0.34279951453208923, 0.6075758934020996, 0.8761252760887146, 1.1488016843795776, 1.425977110862732, 1.708005428314209, 1.9951750040054321, 2.287672281265259, 2.585536003112793, 2.888622522354126, 3.1965713500976562, 3.5087730884552, 3.8243539333343506, 4.142154216766357, 4.460730075836182, 4.778358459472656, 5.09304141998291, 5.4025444984436035, 5.7044196128845215, 5.996051788330078, 6.274717330932617, 6.53762674331665, 6.7819976806640625, 7.005123615264893, 7.204428195953369, 7.3775482177

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test)

In [94]:
for y in y_dict['1']:
    print(y)

Gamble
Gamble
Gamble
Safebet
Gamble
Safebet
Safebet
Safebet
Safebet
Gamble
Safebet
Gamble
Gamble
Gamble
Gamble
Gamble
Gamble
Gamble


In [93]:
y_dict['1']

In [86]:
starter_y = y_dict['1'].tolist()
starter_y.append(starter_y[0])
starter_y

['Gamble',
 'Gamble',
 'Gamble',
 'Safebet',
 'Gamble',
 'Safebet',
 'Safebet',
 'Safebet',
 'Safebet',
 'Gamble',
 'Safebet',
 'Gamble',
 'Gamble',
 'Gamble',
 'Gamble',
 'Gamble',
 'Gamble',
 'Gamble',
 'Gamble']

In [56]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])
c = np.vstack((a,b))
np.vstack((c, b))

array([[1, 2, 3],
       [2, 3, 4],
       [2, 3, 4]])

In [38]:
game_model = pd.read_csv('behavioral_data/gamble_choices.csv')

i = 1
str_int = str(i)
if i < 10:
    str_int = '0' + str_int
tempBehav = pd.read_csv('behavioral_data/gamble.data.s' + str_int + '.csv')
tempNeur = electrode_dict[str(i)]
tempBehav = tempBehav.merge(game_model, left_index=True, right_index=True)

no_bad = tempBehav[badTrials['bad_trials_OFC'][i-1] == 0]
onlyThreshold = no_bad[no_bad['choice.location'] != 'Timeout']

bool_array = [(onlyThreshold['Risky.bet.hidden.number'] == 5).tolist() 
              or (onlyThreshold['Risky.bet.hidden.number'] == 6).tolist() 
              or (onlyThreshold['Risky.bet.hidden.number'] == 7).tolist()]



In [43]:
onlyThreshold[bool_array]

,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.location,choice.class,outcome,Safe.Bet.Amount,Risky.Bet.Amount,Risky.bet.shown.number,Risky.bet.hidden.number,Risky.Side
0,1,6.191,6.957,8.003,8.019,8.535,Left,Gamble,Win,10,30,1,5,Left
1,2,9.550,10.316,11.441,11.457,11.972,Left,Gamble,Win,10,20,4,5,Left
6,7,26.707,27.472,28.753,28.769,29.285,Right,Gamble,Win,10,25,2,5,Right
22,23,85.160,85.925,86.972,86.988,87.503,Left,Safebet,WouldHaveLost,10,25,10,5,Right
38,39,144.863,145.628,146.644,146.675,147.191,Right,Gamble,Win,10,20,4,5,Right
56,57,210.613,211.378,212.894,212.910,213.441,Right,Safebet,WouldHaveWon,10,30,2,5,Left
62,63,231.769,232.535,233.597,233.613,234.144,Left,Safebet,WouldHaveLost,10,20,6,5,Right
71,72,263.363,264.128,265.425,265.441,265.972,Right,Safebet,WouldHaveLost,10,25,6,5,Left
76,77,282.191,282.957,284.191,284.207,284.722,Left,Safebet,WouldHaveLost,10,15,8,5,Right
96,97,354.128,354.894,356.550,356.566,357.082,Right,Gamble,Win,10,25,4,5,Right


In [42]:
electrode_dict['1'][bool_array].shape

(18, 951)

In [7]:
tempNeur = loadmat('ephys_data/s01_ofc_hg_events.mat')['buttonpress_events_hg'][:, 0:951, :]

In [9]:
tempNeur[0, :, :].shape

(951, 5)

In [13]:
len([np.mean(i) for i in tempNeur[0, :, :]])

951

In [15]:
type(tempNeur)

numpy.ndarray

In [94]:
subject_dict = dict()

game_model = pd.read_csv('behavioral_data/gamble_choices.csv')

for i in range(1,10):
    str_int = str(i)
    if i < 10:
        str_int = '0' + str_int
    tempBehav = pd.read_csv('behavioral_data/gamble.data.s' + str_int + '.csv')
    tempNeur = loadmat('ephys_data/s' + str_int + '_ofc_hg_events.mat')
    tempBehav = tempBehav.merge(game_model, left_index=True, right_index=True)

    no_bad = tempBehav[badTrials['bad_trials_OFC'][i-1] == 0]
    onlyThreshold = no_bad[no_bad['choice.location'] != 'Timeout']
    
    
    electrodesBPE = tempNeur['buttonpress_events_hg']
    
    #make columns for electrode feature engineering. use 3 bins
    for j in range(10):
        for b in range(1, 4):
            onlyThreshold.loc[:, 'Electrode' + str(j) + '_min' + '_bin' + str(b)] = np.nan
            onlyThreshold.loc[:, 'Electrode' + str(j) + '_max' + '_bin' + str(b)] = np.nan
            onlyThreshold.loc[:, 'Electrode' + str(j) + '_avg' + '_bin' + str(b)] = np.nan
            
    bin_dict = {'1': range(317), '2': range(317, 634), '3': range(634, 951)}

        
    for e in range(min(electrodesBPE.shape[2], 10)):
        electrode_str = 'Electrode' + str(e)
        for binn in range(1, 4):
            tempBinn = electrodesBPE[:, bin_dict[str(binn)], e]
            #print(len([min(i) for i in tempBinn]))
            #print(len(onlyThreshold.loc[:, electrode_str + '_min' + '_bin' + str(binn)]))
            #print('/n')
            
            onlyThreshold.loc[:, electrode_str + '_min' + '_bin' + str(binn)] = [min(x) for x in tempBinn]
            onlyThreshold.loc[:, electrode_str + '_max' + '_bin' + str(binn)] = [max(x) for x in tempBinn]
            onlyThreshold.loc[:, electrode_str + '_avg' + '_bin' + str(binn)] = [np.mean(x) for x in tempBinn]
            
    #only trials 5 6 7
    
    onlyThreshold = onlyThreshold[(onlyThreshold['Risky.bet.hidden.number'] == 5).tolist()
                          or (onlyThreshold['Risky.bet.hidden.number'] == 6).tolist()
                          or (onlyThreshold['Risky.bet.hidden.number'] == 7).tolist()]
    
    
            
    subject_dict[str(i)] = onlyThreshold
            
    
    
    

In [95]:
for i in range(1, 10):
    print(subject_dict[str(i)].shape)

(18, 104)
(18, 104)
(22, 545)
(9, 104)
(18, 563)
(20, 104)
(18, 113)
(22, 104)
(22, 185)


In [99]:
list(subject_dict['3'])

tempNeur = loadmat('ephys_data/s03_ofc_hg_events.mat')
tempNeur['buttonpress_events_hg'][0, 0, :]


array([  0.55426913,  -0.32384625, -12.948976  ,   4.309811  ,
        11.868046  ,   0.7396951 ,   7.7637253 ,  -0.5958809 ,
         0.09645104,  -6.30518   ,  -2.4209793 ,   2.9704845 ,
         2.454404  ,  -2.6261666 ,   1.1023847 ,  -7.005003  ,
        -0.80429834,   2.1960595 ,  -6.0467095 ,   2.6443324 ,
        -1.975228  ,   2.0067334 ,  -2.2416065 ,  -4.500159  ,
        -6.6233    ,  -9.382331  ,   4.3572016 ,  -0.60849243,
        -2.2989357 ,   4.947932  ,   1.3019736 ,  -5.474989  ,
         2.3415828 ,  -1.0309824 ,  -0.47461882,  -3.584466  ,
        -7.591606  ,   8.451879  ,  -0.19754632,  -2.2670565 ,
         3.571822  ,  11.202229  ,  -0.47461882,   4.9153495 ,
         6.1882153 ,  29.117315  ,  -1.1083817 ,  -3.6007454 ,
        -1.1926463 ,   4.3616986 ,  -6.156525  ,  -4.616151  ,
        29.117315  ,  -1.1083817 ,  -3.6007454 ,  -1.1926463 ,
         4.3616986 ,  -6.156525  ,  -4.616151  ], dtype=float32)

In [100]:
i = 10
str_int = str(i)

tempBehav = pd.read_csv('behavioral_data/gamble.data.s' + str_int + '.csv')
tempNeur = loadmat('ephys_data/s' + str_int + '_ofc_hg_events.mat')
tempBehav = tempBehav.merge(game_model, left_index=True, right_index=True)

no_bad = tempBehav[badTrials['bad_trials_OFC'][i-1][0:138] == 0]
#no_bad = tempBehav[badTrials['bad_trials_OFC'][i-1] == 0]
onlyThreshold = no_bad[no_bad['choice.location'] != 'Timeout']
    

electrodesBPE = tempNeur['buttonpress_events_hg']

print(electrodesBPE.shape)
#make columns for electrode feature engineering. use 3 bins
for j in range(10):
    for b in range(1, 4):
        onlyThreshold.loc[:, 'Electrode' + str(j) + '_min' + '_bin' + str(b)] = np.nan
        onlyThreshold.loc[:, 'Electrode' + str(j) + '_max' + '_bin' + str(b)] = np.nan
        onlyThreshold.loc[:, 'Electrode' + str(j) + '_avg' + '_bin' + str(b)] = np.nan

bin_dict = {'1': range(317), '2': range(317, 634), '3': range(634, 951)}


for e in range(electrodesBPE.shape[2]):
    electrode_str = 'Electrode' + str(e)
    for binn in range(1, 4):
        tempBinn = electrodesBPE[:, bin_dict[str(binn)], e]
        
        onlyThreshold.loc[:, electrode_str + '_min' + '_bin' + str(binn)] = [min(x) for x in tempBinn]
        onlyThreshold.loc[:, electrode_str + '_max' + '_bin' + str(binn)] = [max(x) for x in tempBinn]
        onlyThreshold.loc[:, electrode_str + '_avg' + '_bin' + str(binn)] = [np.mean(x) for x in tempBinn]

#only trials 5 6 7

onlyThreshold = onlyThreshold[(onlyThreshold['Risky.bet.hidden.number'] == 5).tolist()
                      or (onlyThreshold['Risky.bet.hidden.number'] == 6).tolist()
                      or (onlyThreshold['Risky.bet.hidden.number'] == 7).tolist()]



subject_dict[str(i)] = onlyThreshold


(136, 3001, 16)


In [79]:
tempBehav = pd.read_csv('behavioral_data/gamble.data.s10.csv')
tempBehav.shape
tempNeur = loadmat('ephys_data/s' + str_int + '_ofc_hg_events.mat')
tempNeur['buttonpress_events_hg'].shape
onlyThreshold = tempBehav[tempBehav['choice.location'] != 'Timeout']
onlyThreshold.shape

(138, 9)

In [78]:
import os
import pickle
with open(os.path.join('subject_dictno10.pickle'), 'wb') as handle1:
    pickle.dump(subject_dict, handle1, protocol=pickle.HIGHEST_PROTOCOL)

In [101]:
main_frame = subject_dict['1']
main_frame.loc[:, 'subject_number'] = '1'
for key, value in subject_dict.items():
    value.loc[:, 'subject_number'] = key
    main_frame = main_frame.append(value)

    
main_frame.head()

/anaconda3/envs/data-x/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Electrode0_avg_bin1,Electrode0_avg_bin2,Electrode0_avg_bin3,Electrode0_max_bin1,Electrode0_max_bin2,Electrode0_max_bin3,Electrode0_min_bin1,Electrode0_min_bin2,Electrode0_min_bin3,Electrode10_avg_bin1,...,buttonpress.time,choice.class,choice.location,choice.time,conf.time,newround.time,outcome,reveal.time,round,subject_number
0,-1.225318,-4.017107,-1.020703,9.759506,-1.117182,7.899104,-7.119076,-8.019692,-5.910308,NaN,...,8.003,Gamble,Left,6.957,8.019,6.191,Win,8.535,1,1
1,1.255444,-4.530762,-3.141041,31.750257,1.794326,7.294234,-7.611065,-9.168185,-9.389197,NaN,...,11.441,Gamble,Left,10.316,11.457,9.550,Win,11.972,2,1
6,-2.316165,-1.515046,-2.186162,4.166417,6.010494,3.413128,-8.969111,-8.968894,-7.393499,NaN,...,28.753,Gamble,Right,27.472,28.769,26.707,Win,29.285,7,1
22,-0.240522,-2.765261,-1.707884,5.863612,3.445650,3.271917,-6.099566,-5.520102,-5.765873,NaN,...,86.972,Safebet,Left,85.925,86.988,85.160,WouldHaveLost,87.503,23,1
38,2.264580,0.770095,2.598170,8.038921,5.173100,9.414794,-2.865637,-3.738700,-4.209744,NaN,...,146.644,Gamble,Right,145.628,146.675,144.863,Win,147.191,39,1


In [102]:
#main_frame.shape
# some of the subjects have 59 electrodes.... ?? ? ???????

main_frame.to_csv('data567.csv')

In [25]:
game_model = pd.read_csv('behavioral_data/gamble_choices.csv')

s8 = pd.read_csv('behav8_v1.csv')
s8 = s8.merge(game_model, left_index=True, right_index=True)
s8[(s8['Risky.bet.hidden.number']  == 5).tolist() 
   or (s8['Risky.bet.hidden.number']  == 6).tolist()
  or (s8['Risky.bet.hidden.number']  == 7).tolist()].head()


,Unnamed: 0,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.class,choice.location,outcome,...,Electrode_10_avgGE,Electrode_10_minGE,Electrode_10_maxGE,Electrode_10_sdGE,convertedLocation,Safe.Bet.Amount,Risky.Bet.Amount,Risky.bet.shown.number,Risky.bet.hidden.number,Risky.Side
0,0,1,331.856,332.653,334.200,334.216,334.231,Gamble,Left,Win,...,2.771444,-5.182070,42.484066,7.220509,0.0,10,30,1,5,Left
1,1,2,335.263,336.060,337.216,337.247,337.263,Safebet,Right,WouldHaveWon,...,-2.231717,-12.027114,48.651333,9.074130,1.0,10,20,4,5,Left
6,6,7,352.903,353.700,355.731,355.747,355.778,Gamble,Right,Win,...,3.269711,-3.761415,38.507366,6.580947,1.0,10,25,2,5,Right
13,13,14,380.981,381.778,384.278,384.294,384.310,Safebet,Left,WouldHaveWon,...,1.727471,-7.293897,45.554291,7.894190,0.0,10,25,4,5,Right
22,22,23,416.731,417.528,419.997,420.013,420.044,Safebet,Left,WouldHaveLost,...,-4.184577,-10.938680,32.487064,6.916601,0.0,10,25,10,5,Right


In [57]:
tempBehav = pd.read_csv('behavioral_data/gamble.data.s01.csv')
tempNeur = loadmat('ephys_data/s01_ofc_hg_events.mat')
tempBehav = tempBehav.merge(game_model, left_index=True, right_index=True)

no_bad = tempBehav[badTrials['bad_trials_OFC'][i-1] == 0]
no_bad = no_bad[no_bad['choice.location'] != 'Timeout']

#only trials 5 6 7

onlyThreshold = no_bad[(no_bad['Risky.bet.hidden.number'] == 5).tolist()
                          or (no_bad['Risky.bet.hidden.number'] == 6).tolist()
                          or (no_bad['Risky.bet.hidden.number'] == 7).tolist()]
    

In [ ]:
onlyThreshold

In [47]:
bin_dict = {'1': range(317), '2': range(317, 634), '3': range(634, 951)}
bin_dict['1']
test1 = tempNeur['buttonpress_events_hg']
test1[:, 0:317, 0]

array([[-1.5872792 , -1.5010008 , -1.4076289 , ..., -1.4915704 ,
        -1.5727731 , -1.6538423 ],
       [-7.611065  , -7.581905  , -7.537336  , ..., -5.093455  ,
        -5.160134  , -5.1906705 ],
       [-0.62176496, -0.29704818,  0.02554655, ..., -3.3670197 ,
        -3.608842  , -3.822066  ],
       ...,
       [ 3.2969692 ,  3.2126997 ,  3.1295617 , ...,  2.2217865 ,
         2.2307584 ,  2.2272947 ],
       [-7.4678073 , -7.2682014 , -7.0552096 , ..., -7.353522  ,
        -7.2431827 , -7.1116905 ],
       [ 2.752385  ,  2.8246467 ,  2.8881557 , ...,  1.0371097 ,
         1.0556815 ,  1.0659032 ]], dtype=float32)

In [41]:
(s8['Risky.bet.hidden.number']  == 5).tolist() or (s8['Risky.bet.hidden.number']  == 6).tolist()
s8['dummy'] = np.nan
s8.head()

badTrials['bad_trials_OFC'].shape


(10, 200)

In [25]:
np.max(electrodesBPE['Electrode_1'][0, :])

26.868347

In [43]:
badTrials['bad_trials_OFC'][0] == 0

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,

In [53]:
#convert left and right in choice location to 0 and 1 (respectively)
behav8['choice.location'].isnull().any()

behav8['convertedLocation'] = np.nan
for index, row in behav8.iterrows():
    if row['choice.location'] == 'Left':
        behav8.at[index, 'convertedLocation'] = 0
    else:
        behav8.at[index, 'convertedLocation'] = 1
        
behav8.head()
        


,round,newround.time,choice.time,buttonpress.time,conf.time,reveal.time,choice.class,choice.location,outcome,Electrode_1_avgBP,...,Electrode_9_avgGE,Electrode_9_minGE,Electrode_9_maxGE,Electrode_9_sdGE,Electrode_10_sdBP,Electrode_10_avgGE,Electrode_10_minGE,Electrode_10_maxGE,Electrode_10_sdGE,convertedLocation
0,1,331.856,332.653,334.200,334.216,334.231,Gamble,Left,Win,0.626383,...,3.223118,-4.385119,22.050924,5.028584,9.356254,2.771444,-5.182070,42.484066,7.220509,0.0
1,2,335.263,336.060,337.216,337.247,337.263,Safebet,Right,WouldHaveWon,3.069692,...,-2.022406,-10.245128,17.650156,4.604584,7.462085,-2.231717,-12.027114,48.651333,9.074130,1.0
2,3,338.278,339.075,340.450,340.466,340.481,Safebet,Left,WouldHaveWon,-4.822321,...,-4.092527,-15.991223,17.792549,6.787434,9.304393,-2.760110,-12.191625,50.056202,9.161549,0.0
3,4,341.513,342.310,344.247,344.278,344.294,Gamble,Left,Win,-7.254536,...,-1.290316,-11.269731,25.278475,6.205656,12.998384,2.281954,-5.321077,64.502968,9.141026,0.0
4,5,345.310,346.106,347.497,347.513,347.544,Safebet,Right,WouldHaveLost,-2.551188,...,1.484970,-7.191306,22.624266,5.235998,8.202618,0.515470,-12.807647,83.155434,14.008977,1.0


In [54]:
from sklearn.model_selection import train_test_split

X = behav8.drop(['choice.class', 'outcome', 'choice.location'], axis = 1)
Y = behav8['outcome']



x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=100)


In [55]:
from sklearn import linear_model

# Name our logistic regression object
LogisticRegressionModel = linear_model.LogisticRegression()

# we create an instance of logistic Regression Classifier and fit the data.
print ('Training a logistic Regression Model..')
LogisticRegressionModel.fit(x_train, y_train)

training_accuracy=LogisticRegressionModel.score(x_train,y_train)
print ('Training Accuracy: ', training_accuracy)

Training a logistic Regression Model..
('Training Accuracy: ', 0.97499999999999998)


In [56]:
test_accuracy=LogisticRegressionModel.score(x_test,y_test)
print('Accuracy of the model on unseen test data: ',test_accuracy)

('Accuracy of the model on unseen test data: ', 0.42499999999999999)


In [57]:
#serious overfitting in the above. 
# going to check it out in r...
behav8.to_csv('behav8_v1.csv')

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import mne
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
import pandas as pd


In [3]:
 best_trials = pd.read_csv('best_trials_master_df.csv')

In [8]:
neur8 = loadmat('s08_ofc_hg_events.mat')
neur8

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Wed Oct  4 10:18:12 2017',
 '__version__': '1.0',
 '__globals__': [],
 'buttonpress_events_hg': array([[[  8.189263  ,  -3.3722084 ,  -2.2066743 , ...,  -5.42372   ,
           19.079905  ,   0.42457733],
         [  8.309072  ,  -3.7229521 ,  -2.248763  , ...,  -5.406721  ,
           18.957127  ,   0.6265146 ],
         [  8.381103  ,  -4.033373  ,  -2.2611573 , ...,  -5.436369  ,
           18.791298  ,   0.8089374 ],
         ...,
         [ -0.3490263 ,   3.5280123 ,  23.265627  , ...,   3.3741572 ,
           10.448634  ,   8.574485  ],
         [  0.27512226,   3.8945847 ,  22.758974  , ...,   4.1000443 ,
           10.329502  ,   8.384635  ],
         [  0.9248714 ,   4.243525  ,  22.263474  , ...,   4.8184342 ,
           10.1563015 ,   8.156524  ]],
 
        [[  8.282565  ,   9.261214  ,  16.951536  , ...,   9.476936  ,
           -0.24441488,   0.36490318],
         [  8.261956  ,   9.516955  ,  16.579891  

In [139]:


# try the same with result_frame ?

result_frame = pd.read_csv('result_frame.csv')
result_frame.head()



,Unnamed: 0,Subject,Time Series,Max,Min,Interval,Std,p2.5,p97.5,Percentile Interval,Num Peaks,Num Peaks Cat,Result
0,0,1,"[-3.7782058715820312, -3.9974231719970703, -4....",6.990900,-5.175566,12.166465,2.641384,-4.395109,5.985634,10.380744,16,2,Safebet
1,1,1,"[1.0019606351852417, 0.737346351146698, 0.4520...",10.671882,-5.144136,15.816019,3.041047,-4.143518,7.848766,11.992284,15,2,Gamble
2,2,1,"[5.230727195739746, 5.047644138336182, 4.87180...",5.877920,-5.293972,11.171892,2.428461,-4.761420,4.982319,9.743739,20,3,Gamble
3,3,1,"[6.716716289520264, 6.5874199867248535, 6.4329...",8.061066,-6.061206,14.122271,3.001979,-5.291393,7.276974,12.568367,15,2,Gamble
4,4,1,"[-4.0423502922058105, -4.200969219207764, -4.3...",9.490667,-5.644985,15.135653,2.791258,-4.919451,6.252715,11.172166,19,3,Gamble


In [151]:
result_frame2 = result_frame.drop(['Subject'], axis = 1)
result_frame2 = result_frame2.drop('Unnamed: 0', axis = 1)

result_frame2.head()
X_data = result_frame2.drop('Result', axis = 1)
y_data = result_frame2['Result']